In [2]:
import json
import pandas as pd

json_file = '../01_scraping/json/espn_2022calendar.json'
df = pd.read_json(json_file)

#df_wk1 = df.loc[(df.week==1), ["awayteam","hometeam"]]
#df_wk1

df = df.replace("Commanders", "Washington")

#df_wk1.to_csv('../04_datasets/nfl2022_week1.csv')
df.to_csv('../04_datasets/nfl2022_total.csv')